# Flight Streaming Data

This directory has code and schemas for interacting with the streaming data from OpenSky.

The python/flight/stream folder contains code that will pull the latest data from OpenSky and store the data in GCS and/or publish the data as messages in a Pub/Sub queue.

## How to Run Locally

To execute the Python code, you will need to follow the instructions in the README (displayed in the main page in the GitHub repository.) You only need to set up your Python environment if you want to run the code from Cloud Shell (to test it out without using Cloud Function.)

## How to Run Serveless

To run using a Cloud Function (aka. using serverless computing) you supply the code and a list of required libraries to Cloud Function, connect the function to Cloud Storage or a Pub/Sub queue, and provide the function with permission to access your Cloud Storage bucket or your Pub/Sub queue. You can then create a Big Query table from either the Cloud Storage bucket or the Pub/Sub queue using the provided schemas.

## Schemas

The schema folder has schemas used both for BigQuery tables as well as for connecting a Pub/Sub topic to a BigQuery table.
BigQuery schemas are used when you create a table and do not select to autodetect the schema. 

To create a table as an endpoint for messages published in a Pub/Sub topic will also need a schema that defines how to create columns out of the contents of the messages. The schema needed by DataFlow uses different types names than BigQuery.

**schema/openSky_bigQuery.json** -- a schema that can be used when loading data from Cloud Storage.

**schema/openSky_dataFlow.json** -- a schema that defines how to map the fields in a Pub/Sub message to columns.